<a href="https://colab.research.google.com/github/hassanalubeidi/RAG/blob/main/Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

from openai import OpenAI

In [ ]:
from google.colab import userdata
openai_key=userdata.get('openai-key')

client = OpenAI(api_key=openai_key)

resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You will answer in 1 sentance"},
        {
            "role": "user",
            "content": "What is UBS's 2024 Q3 pre tax profit?",
        },
    ],
)

resp.choices[0].message.content

"I don't have access to real-time data, so I can't provide UBS's 2024 Q3 pre-tax profit."

In [ ]:
import json
resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You will answer in 1 sentance"},
        {
            "role": "user",
            "content": "What is UBS's 2024 Q3 pre tax profit?",
        },
    ],
    tools=[{
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
                },
                "required": ["query"]
            }
        }
    }]
)

print(resp)
tool_call_resp = resp.choices[0].message.tool_calls[0].function.arguments
print(tool_call_resp)
tool_call = json.loads(tool_call_resp)
tool_call

ChatCompletion(id='chatcmpl-AUiuhCy6Jgs3snsMHlxnLnmf85jNJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ytelcFfiNM5t2nD6oXjCAGUg', function=Function(arguments='{"query":"UBS 2024 Q3 pre tax profit"}', name='tavily_search'), type='function')]))], created=1731885851, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_831e067d82', usage=CompletionUsage(completion_tokens=25, prompt_tokens=87, total_tokens=112, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
{"query":"UBS 2024 Q3 pre tax profit"}


{'query': 'UBS 2024 Q3 pre tax profit'}

In [ ]:
!pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00


In [ ]:


from tavily import TavilyClient
tavily_client = TavilyClient(userdata.get('tavily-key'))
tavily_results = tavily_client.get_search_context(tool_call["query"])
tavily_results

'"[\\"{\\\\\\"url\\\\\\": \\\\\\"https://www.morningstar.com/news/marketwatch/20241030167/ubs-smashes-forecasts-with-bumper-profit-warns-of-volatility-around-us-elections\\\\\\", \\\\\\"content\\\\\\": \\\\\\"The Swiss bank beat forecasts in posting a 31% increase in its pre-tax profits, to $1.9 billion, in the third quarter of 2024, versus the $1.7 billion forecast by 15 analysts polled by UBS itself.\\\\\\"}\\", \\"{\\\\\\"url\\\\\\": \\\\\\"https://www.ubs.com/global/en/media/display-page-ndp/en-20241030-3q24-quarterly-result.html\\\\\\", \\\\\\"content\\\\\\": \\\\\\"UBS\'s third-quarter 2024 results | UBS Global UBS\'s third-quarter 2024 results UBS reports 3Q24 net profit of USD 1.4bn with 6.2trn in invested assets; delivering for clients, executing on integration at pace and investing for growth Continued client momentum with USD 25bn of net new assets in Global Wealth Management, on track to deliver on our ambition of USD ~100bn in NNA for 2024; Group invested assets of USD 6.2

In [ ]:
resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You will answer in 1 sentance"},
        {"role": "user","content": f"context: {str(tavily_results)}"},
        {
            "role": "user",
            "content": "What is UBS's 2024 Q3 pre tax profit?",
        },
    ],
    tools=[{
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
                },
                "required": ["query"]
            }
        }
    }]
)

resp.choices[0].message.content

"UBS's 2024 Q3 pre-tax profit is $1.9 billion."